In [1]:
pip install -q peft transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.7 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 

In [2]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 22.2 MB/s eta 0:00:0000:01:00:01


In [3]:
!pip install -q trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 9.7 MB/s eta 0:00:00:00:01


In [4]:
!pip install sentencepiece

In [40]:
import torch
from transformers import (
AutoTokenizer,
AutoModelForCausalLM,
BitsAndBytesConfig,
pipeline,
logging,
HfArgumentParser,
TrainingArguments
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import os

In [5]:
!pip install -q -U einops

In [43]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [44]:
logging.set_verbosity_warning()

In [45]:
model_name = 'NousResearch/Llama-2-7b-chat-hf'

In [46]:
dataset_name = 'chrishayuk/test'

In [47]:
new_model = 'llama-2-7b-chuk-test'

In [48]:
output_dir = '/kaggle/working/'

In [49]:
num_train_epochs = 20

In [50]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = False,
)

In [51]:
torch.cuda.empty_cache()

In [52]:
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config = bnb_config,
    device_map = {"": 0}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [53]:
model.config.use_cache = False

In [54]:
model.config.pretraining_tp = 1

In [55]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [56]:
prompt = 'Write a hello world program in the OPL programming language.'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 200)
result = pipe(f'<s>[INST] {prompt} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<s>[INST] Write a hello world program in the OPL programming language. [/INST]  Sure! Here is an example of a "Hello World" program in the OPL programming language:
```
PRINT "Hello, World!"
```
This program will output the string "Hello, World!" to the console when run.

Here's a breakdown of the syntax:

* `PRINT`: This keyword is used to print a value to the console.
* `":"`: This is the colon used to separate the value from the rest of the program.
* `"Hello, World!"`: This is the value that will be printed to the console.

That's it! This is the basic syntax of an OPL program. Of course, there are many more features and capabilities of the OPL language that you can explore as you learn more about it.


In [57]:
torch.cuda.empty_cache()

In [58]:
dataset = load_dataset(dataset_name, split = 'train')

In [59]:
dataset[0]

{'text': '<s>[INST] What does OPL stand for in the OPL programming language? [/INST] OPL is short for Open Programming Language </s>'}

In [60]:
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=512
    )

In [61]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [63]:
peft_config = LoraConfig(
    lora_alpha = 16,
    r = 64,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

In [64]:
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    optim = 'paged_adamw_8bit',
    save_steps = 0,
    logging_steps = 10,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    fp16 = True,
    bf16 = False,
    max_grad_norm = 0.3,
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type = 'cosine',
    report_to = 'tensorboard'
)

In [65]:
trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_dataset,
    peft_config = peft_config,
    args = training_arguments,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [66]:
torch.cuda.empty_cache()

In [69]:
print("Starting training...")
trainer.train()
print("Training complete!")

Starting training...


Step,Training Loss
10,0.105500
20,0.079600
30,0.052000
40,0.040100


Training complete!


In [70]:
trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('llama-2-7b-chuk-test/tokenizer_config.json',
 'llama-2-7b-chuk-test/special_tokens_map.json',
 'llama-2-7b-chuk-test/tokenizer.model',
 'llama-2-7b-chuk-test/added_tokens.json')

In [71]:
print(f"Model and tokenizer saved to {new_model}")

Model and tokenizer saved to llama-2-7b-chuk-test


In [72]:
prompt = 'Write a tax calculator in OPL'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 200)
result = pipe(f'<s>[INST] {prompt} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Write a tax calculator in OPL [/INST] PROC main:
	LOCAL percentage%
	LOCAL income%
	LOCAL tax%
	PRINT "Tax Calculator\n"
	PRINT "Enter your income:"
	INPUT income%
	PRINT "Enter the tax rate (%):"
	INPUT percentage%
	tax% = (income% * percentage%) / 100.0
	PRINT "Tax:", tax%
	GET
ENDP 


In [75]:
prompt = 'Write a Hello Chris program in psion opl'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 200)
result = pipe(f'<s>[INST] {prompt} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Write a Hello Chris program in psion opl [/INST] PROC Hello:
	LOCAL name$(10)
	PRINT "Hello ", name$
	GET
ENDP 
